In [148]:
import numpy as np
import pandas as pd
import math as math
%matplotlib inline
import matplotlib.pyplot as plt

data = pd.read_csv('data2.csv', header=None).as_matrix()
print(data.shape)

(5000, 2)


In [149]:
# calculate odds ratio for each feature given C = 0, then odds for each feature given C = 1
# split data into X and Y
def split_data(data): 
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

X, y = split_data(data)


In [150]:
# calculate odds ratio for each feature given C = 0
# create vector of T/F for all features whose C = 0
class_zero_data = X[y == 0, :]
class_one_data = X[y == 1, :]

count_in_class_zero = class_zero_data.shape[0]
count_in_class_one = class_one_data.shape[0]

feature_presence_probs_in_class_zero = (np.sum(class_zero_data, axis=0) + 1) / (count_in_class_zero + 1)
feature_absence_probs_in_class_zero = (count_in_class_zero - np.sum(class_zero_data, axis=0) + 1) / (count_in_class_zero + 1)
feature_presence_probs_in_class_one = (np.sum(class_one_data, axis=0) + 1) / (count_in_class_one + 1)
feature_absence_probs_in_class_one = (count_in_class_one - np.sum(class_one_data, axis=0) + 1) / (count_in_class_one + 1)

In [151]:
log_odds = np.zeros((X.shape[1], 2))
log_odds[:, 0] = np.log(feature_absence_probs_in_class_one / feature_absence_probs_in_class_zero)
log_odds[:, 1] = np.log(feature_presence_probs_in_class_one / feature_presence_probs_in_class_zero)

p_c0 = count_in_class_zero / X.shape[0]
p_c1 = count_in_class_one / X.shape[0]
base_log_odds = np.log((p_c1 + 1) / (p_c0 + 1))
print(log_odds)
print(base_log_odds)

def predict(log_odds, base_log_odds, x):
    z = np.dot(x, log_odds[:, 1]) + np.dot(1 - x, log_odds[:, 0]) + base_log_odds
    return 1 / (1 + np.exp(-z))

[[ 0.89350901 -1.17815718]]
0.00560001463474


In [152]:
def calc_accuracy(X, y, log_odds, base_odds):
    predictions = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        prob_c_one = predict(log_odds, base_odds, X[i])
        if prob_c_one > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0
    return np.sum(y == predictions) / y.shape[0]

print(calc_accuracy(X, y, log_odds, base_log_odds))


0.7346
